# Search and Download Demo Notebook

This demo notebook walks through the steps for searching and downloading PlanetScope imagery using the planetsca library.

You will need a user account with Planet, and your own [API key](https://developers.planet.com/quickstart/apis/#find-your-api-key) to use these functions.

In [ ]:
import planetsca as ps

You will need an [API key](https://developers.planet.com/quickstart/apis/#find-your-api-key) for searching and downloading Planet imagery.

In [ ]:
api_key = ""

Set up a filter to search a specific time range and region, and to avoid cloudy images:

In [ ]:
# filter images acquired in a certain date range
date_range_filter = ps.search.make_date_range_filter(
    "2024-01-15T00:00:00.000Z", "2024-01-31T00:00:00.000Z"
)

There are two options for specifying a region to search: creating a "geometry filter" from lat and lon bounding coordinates, or creating a "geometry filter" from a geojson file.

In [ ]:
# Example for defining an area of interest from bounds
minLon = -105.88
minLat = 40.51
maxLon = -105.87
maxLat = 40.52

# make a geometry filter with these bounding coordinates
geometry_filter = ps.search.make_geometry_filter_from_bounds(
    [minLon, minLat, maxLon, maxLat]
)


# Alternatively, define area of interest from a geojson file
# geometry_filter = ps.search.make_geometry_filter_from_geojson('my_study_area.geojson')

Add a cloud cover percentage filter

In [ ]:
cloud_filter = ps.search.make_cloud_cover_filter(
    0.05
)  # filter any images which are more than 5% clouds

Finally, create a filter that combines our geometry, date, and cloud filters

In [ ]:
filter = ps.search.combine_filters([date_range_filter, geometry_filter, cloud_filter])

Submit our search to the Planet API. This returns a geopandas.GeoDataFrame with information about the images that match our filter criteria

In [ ]:
search_results_gdf = ps.search.search(api_key, filter)

In [ ]:
search_results_gdf

Perhaps we'd like to exclude images that overlap our region of interest by less than 50%, we can do that with the following:

In [ ]:
search_results_gdf.drop(
    search_results_gdf[search_results_gdf.overlap_percentage < 50].index, inplace=True
)

In [ ]:
search_results_gdf

We can also save the results of this search to a csv file for later reference.

In [ ]:
search_results_gdf.to_csv("my_images.csv")

To submit an order to the Planet API for the images we want, we will need to grab the item IDs from one column in the search results GeoDataFrame.

In [ ]:
id_list = search_results_gdf.id.to_list()  # get item IDs as a list
id_list

Now, submit the order:

In [ ]:
order_url = ps.download.order(api_key, id_list, filter)

The `order_url` returned by this function is where we'll download our images from.

The download function below will try to download the images, but if they're not yet ready, it will wait 60 seconds before tyring again. It can take about 3-5 minutes for an order of about 10 images to be prepared before being available for download.

In [ ]:
# specify the directory you'd like to save your downloaded images to
out_dirpath = "../demo_images"

ps.download.download(api_key, order_url, out_dirpath="../demo_images")